In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, explode, col, regexp_replace
from pyspark.sql.types import StringType

In [2]:
# Create a SparkSession
spark = SparkSession.builder.appName("CleanStackedHeader").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/07 17:25:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
# Read the data as a string type
df = spark.read.text("clinicaltrial_2023.csv")  

In [4]:
display(df.head(10))

[Row(value='"Id\tStudy Title\tAcronym\tStatus\tConditions\tInterventions\tSponsor\tCollaborators\tEnrollment\tFunder Type\tType\tStudy Design\tStart\tCompletion",,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,'),
 Row(value='"NCT03630471\tEffectiveness of a Problem-solving Intervention for Common Adolescent Mental Health Problems in India\tPRIDE\tCOMPLETED\tMental Health Issue (E.G.", Depression, Psychosis, Personality Disorder," Substance Abuse)\tBEHAVIORAL: PRIDE \'Step 1\' problem-solving intervention|BEHAVIORAL: Enhanced usual care\tSangath\tHarvard Medical School (HMS and HSDM)|London School of Hygiene and Tropical Medicine\t250.0\tOTHER\tINTERVENTIONAL\tAllocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: DOUBLE (INVESTIGATOR"," OUTCOMES_ASSESSOR)|Primary Purpose: TREATMENT\t2018-08-20\t2019-02-28",,,,,,,,,,,,,,,

In [5]:
from pyspark.sql import functions as F

df = df.withColumn('value', F.regexp_replace(F.col('value'), ',+$', ''))

In [6]:
for col_name in df.columns:
    df = df.withColumn(col_name, F.regexp_replace(F.col(col_name), ',+$', ''))

In [7]:
df.toPandas().head(10)

,value
0,"""Id\tStudy Title\tAcronym\tStatus\tConditions\..."
1,"""NCT03630471\tEffectiveness of a Problem-solvi..."
2,"""NCT05992571\tOral Ketone Monoester Supplement..."
3,"""NCT00237471\tImpact of Tight Glycaemic Contro..."
4,"""NCT03820271\tNew Prognostic Predictive Models..."
5,"""NCT06229171\tInTake Care: Development and Val..."
6,"""NCT02945371\tTailored Inhibitory Control Trai..."
7,"""NCT01055171\tNeuromodulation of Trauma Memori..."
8,"""NCT01125371\tComputerized Brief Alcohol Inter..."
9,"""NCT02554071\tManitoba Pharmacist Initiated Sm..."


In [8]:
df = df.select(F.split(df['value'], '\t').alias('split_value'))

df = df.select([df['split_value'][i].alias(f'col_{i}') for i in range(len(df.select('split_value').first()[0]))])

df = df.drop('value')

In [9]:
df.toPandas().head(10)

,col_0,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13
0,"""Id",Study Title,Acronym,Status,Conditions,Interventions,Sponsor,Collaborators,Enrollment,Funder Type,Type,Study Design,Start,"Completion"""
1,"""NCT03630471",Effectiveness of a Problem-solving Interventio...,PRIDE,COMPLETED,"Mental Health Issue (E.G."", Depression, Psycho...",BEHAVIORAL: PRIDE 'Step 1' problem-solving int...,Sangath,Harvard Medical School (HMS and HSDM)|London S...,250.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2018-08-20,"2019-02-28"""
2,"""NCT05992571",Oral Ketone Monoester Supplementation and Rest...,,RECRUITING,Cerebrovascular Function|Cognition,OTHER: Placebo|DIETARY_SUPPLEMENT: β-OHB,McMaster University,"Alzheimer's Society of Brant"", Haldimand Norfo...",30.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: CRO...,2023-10-25,"2024-08"""
3,"""NCT00237471",Impact of Tight Glycaemic Control in Acute Myo...,,TERMINATED,Myocardial Infarct|Hyperglycemia,DRUG: Insulin (tight blood glucose control),Melbourne Health,"National Health and Medical Research Council"",...",40.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2005-10,"2006-05"""
4,"""NCT03820271",New Prognostic Predictive Models of Mortality ...,SUPERMELD,RECRUITING,Decompensated Cirrhosis|Liver Transplantation,OTHER: SuperMELD,Assistance Publique - Hôpitaux de Paris,,500.0,OTHER,INTERVENTIONAL,Allocation: NA|Intervention Model: SINGLE_GROU...,2020-10-01,"2023-10-01"""
5,"""NCT06229171",InTake Care: Development and Validation of an ...,InTakeCare,NOT_YET_RECRUITING,Hypertension|Treatment Adherence and Complianc...,OTHER: adherence support system based on a voc...,Istituto Auxologico Italiano,Istituti Clinici Scientifici Maugeri SpA|Polit...,206.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2024-10-01,"2026-04-01"""
6,"""NCT02945371",Tailored Inhibitory Control Training to Revers...,REV,COMPLETED,Smoking|Alcohol Drinking|Prescription Drug Abu...,BEHAVIORAL: Person-centered inhibitory control...,University of Oregon,,103.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2014-09,"2016-05"""
7,"""NCT01055171",Neuromodulation of Trauma Memories in PTSD & A...,,COMPLETED,Alcohol Dependence|PTSD,DRUG: Propranolol|DRUG: Placebo,Medical University of South Carolina,National Institute on Alcohol Abuse and Alcoho...,44.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2010-01,"2012-08"""
8,"""NCT01125371",Computerized Brief Alcohol Intervention (BI) f...,,COMPLETED,Alcohol; Harmful Use|Binge Drinking|Risk Behav...,BEHAVIORAL: Computerized brief alcohol interve...,Johns Hopkins University,National Institute on Alcohol Abuse and Alcoho...,439.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2011-10,"2016-06-07"""
9,"""NCT02554071",Manitoba Pharmacist Initiated Smoking Cessatio...,,COMPLETED,Smoking Cessation,OTHER: Pharmacist - Smoking Cessation Support,University of Manitoba,Govenment of Manitoba|Canadian Foundation for ...,119.0,OTHER,INTERVENTIONAL,Allocation: NA|Intervention Model: SINGLE_GROU...,2014-01,"2014-11"""


In [10]:
for col_name in df.columns:
    null_count = df.filter(F.col(col_name).isNull()).count()
    print(f'Column {col_name} has {null_count} null values.')

Column col_0 has 0 null values.


Column col_1 has 0 null values.


Column col_2 has 0 null values.


Column col_3 has 0 null values.


Column col_4 has 0 null values.


...

Column col_9 has 2 null values.


Column col_10 has 2 null values.


Column col_11 has 2 null values.


Column col_12 has 2 null values.


Column col_13 has 2 null values.


In [11]:

new_column_names = df.first()  
for col in df.columns:
    df = df.withColumnRenamed(col, new_column_names[col])


In [12]:
df.toPandas().head(10)

,"""Id",Study Title,Acronym,Status,Conditions,Interventions,Sponsor,Collaborators,Enrollment,Funder Type,Type,Study Design,Start,"Completion"""
0,"""Id",Study Title,Acronym,Status,Conditions,Interventions,Sponsor,Collaborators,Enrollment,Funder Type,Type,Study Design,Start,"Completion"""
1,"""NCT03630471",Effectiveness of a Problem-solving Interventio...,PRIDE,COMPLETED,"Mental Health Issue (E.G."", Depression, Psycho...",BEHAVIORAL: PRIDE 'Step 1' problem-solving int...,Sangath,Harvard Medical School (HMS and HSDM)|London S...,250.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2018-08-20,"2019-02-28"""
2,"""NCT05992571",Oral Ketone Monoester Supplementation and Rest...,,RECRUITING,Cerebrovascular Function|Cognition,OTHER: Placebo|DIETARY_SUPPLEMENT: β-OHB,McMaster University,"Alzheimer's Society of Brant"", Haldimand Norfo...",30.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: CRO...,2023-10-25,"2024-08"""
3,"""NCT00237471",Impact of Tight Glycaemic Control in Acute Myo...,,TERMINATED,Myocardial Infarct|Hyperglycemia,DRUG: Insulin (tight blood glucose control),Melbourne Health,"National Health and Medical Research Council"",...",40.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2005-10,"2006-05"""
4,"""NCT03820271",New Prognostic Predictive Models of Mortality ...,SUPERMELD,RECRUITING,Decompensated Cirrhosis|Liver Transplantation,OTHER: SuperMELD,Assistance Publique - Hôpitaux de Paris,,500.0,OTHER,INTERVENTIONAL,Allocation: NA|Intervention Model: SINGLE_GROU...,2020-10-01,"2023-10-01"""
5,"""NCT06229171",InTake Care: Development and Validation of an ...,InTakeCare,NOT_YET_RECRUITING,Hypertension|Treatment Adherence and Complianc...,OTHER: adherence support system based on a voc...,Istituto Auxologico Italiano,Istituti Clinici Scientifici Maugeri SpA|Polit...,206.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2024-10-01,"2026-04-01"""
6,"""NCT02945371",Tailored Inhibitory Control Training to Revers...,REV,COMPLETED,Smoking|Alcohol Drinking|Prescription Drug Abu...,BEHAVIORAL: Person-centered inhibitory control...,University of Oregon,,103.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2014-09,"2016-05"""
7,"""NCT01055171",Neuromodulation of Trauma Memories in PTSD & A...,,COMPLETED,Alcohol Dependence|PTSD,DRUG: Propranolol|DRUG: Placebo,Medical University of South Carolina,National Institute on Alcohol Abuse and Alcoho...,44.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2010-01,"2012-08"""
8,"""NCT01125371",Computerized Brief Alcohol Intervention (BI) f...,,COMPLETED,Alcohol; Harmful Use|Binge Drinking|Risk Behav...,BEHAVIORAL: Computerized brief alcohol interve...,Johns Hopkins University,National Institute on Alcohol Abuse and Alcoho...,439.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2011-10,"2016-06-07"""
9,"""NCT02554071",Manitoba Pharmacist Initiated Smoking Cessatio...,,COMPLETED,Smoking Cessation,OTHER: Pharmacist - Smoking Cessation Support,University of Manitoba,Govenment of Manitoba|Canadian Foundation for ...,119.0,OTHER,INTERVENTIONAL,Allocation: NA|Intervention Model: SINGLE_GROU...,2014-01,"2014-11"""


In [13]:
from pyspark.sql.functions import regexp_replace, col

for col_name in df.columns:
    df = df.withColumn(col_name, regexp_replace(col(col_name), '"', ''))

In [14]:
df.toPandas()

,"""Id",Study Title,Acronym,Status,Conditions,Interventions,Sponsor,Collaborators,Enrollment,Funder Type,Type,Study Design,Start,"Completion"""
0,Id,Study Title,Acronym,Status,Conditions,Interventions,Sponsor,Collaborators,Enrollment,Funder Type,Type,Study Design,Start,Completion
1,NCT03630471,Effectiveness of a Problem-solving Interventio...,PRIDE,COMPLETED,"Mental Health Issue (E.G., Depression, Psychos...",BEHAVIORAL: PRIDE 'Step 1' problem-solving int...,Sangath,Harvard Medical School (HMS and HSDM)|London S...,250.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2018-08-20,2019-02-28
2,NCT05992571,Oral Ketone Monoester Supplementation and Rest...,,RECRUITING,Cerebrovascular Function|Cognition,OTHER: Placebo|DIETARY_SUPPLEMENT: β-OHB,McMaster University,"Alzheimer's Society of Brant, Haldimand Norfol...",30.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: CRO...,2023-10-25,2024-08
3,NCT00237471,Impact of Tight Glycaemic Control in Acute Myo...,,TERMINATED,Myocardial Infarct|Hyperglycemia,DRUG: Insulin (tight blood glucose control),Melbourne Health,"National Health and Medical Research Council, ...",40.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2005-10,2006-05
4,NCT03820271,New Prognostic Predictive Models of Mortality ...,SUPERMELD,RECRUITING,Decompensated Cirrhosis|Liver Transplantation,OTHER: SuperMELD,Assistance Publique - Hôpitaux de Paris,,500.0,OTHER,INTERVENTIONAL,Allocation: NA|Intervention Model: SINGLE_GROU...,2020-10-01,2023-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483418,NCT02975895,Outcome of Different IOLs in Patients With and...,,COMPLETED,Uveitis|Cataract|Inflammation|Posterior Capsul...,DEVICE: Hydrophobic IOL: Vivinex (HOYA)|DEVICE...,Vastra Gotaland Region,,56.0,OTHER_GOV,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2016-12-30,2022-03-17
483419,NCT03332095,"Evaluating the Pharmacokinetics, Safety, and T...",,COMPLETED,HIV Infections,DRUG: Doravirine (DOR)|DRUG: Doravirine/Lamivu...,National Institute of Allergy and Infectious D...,,55.0,NIH,INTERVENTIONAL,Allocation: NON_RANDOMIZED|Intervention Model:...,2018-07-02,2022-05-25
483420,NCT05191095,The Accuracy of Human Endoscopic Detection of ...,,COMPLETED,Colorectal Polyp,,"University Hospital, Ghent",,82.0,OTHER,OBSERVATIONAL,Observational Model: |Time Perspective: p,2021-03-01,2021-06-01
483421,NCT01388595,Fluticasone and Salmeterol in Allergic Rhinitis,,COMPLETED,Allergic Rhinitis,DRUG: FLUTICASONE PROPRIONATE|DRUG: Salmeterol...,University of Dundee,,23.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: CRO...,2006-11,2010-10


In [15]:
from pyspark.sql import Row

first_row = df.first()

df_new = df.filter(df['"Id'] != first_row['"Id'])

header = first_row.asDict()


for col_name in df_new.columns:
    df_new = df_new.withColumnRenamed(col_name, header[col_name])

df = df_new


In [16]:
df.toPandas()

,Id,Study Title,Acronym,Status,Conditions,Interventions,Sponsor,Collaborators,Enrollment,Funder Type,Type,Study Design,Start,Completion
0,NCT03630471,Effectiveness of a Problem-solving Interventio...,PRIDE,COMPLETED,"Mental Health Issue (E.G., Depression, Psychos...",BEHAVIORAL: PRIDE 'Step 1' problem-solving int...,Sangath,Harvard Medical School (HMS and HSDM)|London S...,250.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2018-08-20,2019-02-28
1,NCT05992571,Oral Ketone Monoester Supplementation and Rest...,,RECRUITING,Cerebrovascular Function|Cognition,OTHER: Placebo|DIETARY_SUPPLEMENT: β-OHB,McMaster University,"Alzheimer's Society of Brant, Haldimand Norfol...",30.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: CRO...,2023-10-25,2024-08
2,NCT00237471,Impact of Tight Glycaemic Control in Acute Myo...,,TERMINATED,Myocardial Infarct|Hyperglycemia,DRUG: Insulin (tight blood glucose control),Melbourne Health,"National Health and Medical Research Council, ...",40.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2005-10,2006-05
3,NCT03820271,New Prognostic Predictive Models of Mortality ...,SUPERMELD,RECRUITING,Decompensated Cirrhosis|Liver Transplantation,OTHER: SuperMELD,Assistance Publique - Hôpitaux de Paris,,500.0,OTHER,INTERVENTIONAL,Allocation: NA|Intervention Model: SINGLE_GROU...,2020-10-01,2023-10-01
4,NCT06229171,InTake Care: Development and Validation of an ...,InTakeCare,NOT_YET_RECRUITING,Hypertension|Treatment Adherence and Complianc...,OTHER: adherence support system based on a voc...,Istituto Auxologico Italiano,Istituti Clinici Scientifici Maugeri SpA|Polit...,206.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2024-10-01,2026-04-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
483417,NCT02975895,Outcome of Different IOLs in Patients With and...,,COMPLETED,Uveitis|Cataract|Inflammation|Posterior Capsul...,DEVICE: Hydrophobic IOL: Vivinex (HOYA)|DEVICE...,Vastra Gotaland Region,,56.0,OTHER_GOV,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: PAR...,2016-12-30,2022-03-17
483418,NCT03332095,"Evaluating the Pharmacokinetics, Safety, and T...",,COMPLETED,HIV Infections,DRUG: Doravirine (DOR)|DRUG: Doravirine/Lamivu...,National Institute of Allergy and Infectious D...,,55.0,NIH,INTERVENTIONAL,Allocation: NON_RANDOMIZED|Intervention Model:...,2018-07-02,2022-05-25
483419,NCT05191095,The Accuracy of Human Endoscopic Detection of ...,,COMPLETED,Colorectal Polyp,,"University Hospital, Ghent",,82.0,OTHER,OBSERVATIONAL,Observational Model: |Time Perspective: p,2021-03-01,2021-06-01
483420,NCT01388595,Fluticasone and Salmeterol in Allergic Rhinitis,,COMPLETED,Allergic Rhinitis,DRUG: FLUTICASONE PROPRIONATE|DRUG: Salmeterol...,University of Dundee,,23.0,OTHER,INTERVENTIONAL,Allocation: RANDOMIZED|Intervention Model: CRO...,2006-11,2010-10


In [17]:
for col_name in df.columns:
    null_count = df.filter(F.col(col_name).isNull()).count()
    print(f'Column {col_name} has {null_count} null values.')

Column Id has 0 null values.


Column Study Title has 0 null values.


Column Acronym has 0 null values.


Column Status has 0 null values.


Column Conditions has 0 null values.


...

Column Funder Type has 2 null values.


Column Type has 2 null values.


Column Study Design has 2 null values.


Column Start has 2 null values.


Column Completion has 2 null values.


In [18]:
print(f"Schema of the dataframe: ")
df.printSchema()

Schema of the dataframe: 
root
 |-- Id: string (nullable = true)
 |-- Study Title: string (nullable = true)
 |-- Acronym: string (nullable = true)
 |-- Status: string (nullable = true)
 |-- Conditions: string (nullable = true)
 |-- Interventions: string (nullable = true)
 |-- Sponsor: string (nullable = true)
 |-- Collaborators: string (nullable = true)
 |-- Enrollment: string (nullable = true)
 |-- Funder Type: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Study Design: string (nullable = true)
 |-- Start: string (nullable = true)
 |-- Completion: string (nullable = true)



In [19]:
for col in df.columns:
    null_rows = df.filter(F.col(col).isNull()).collect()
    print(f"Null valued rows in {col}: {null_rows}\n \n")

Null valued rows in Id: []
 



Null valued rows in Study Title: []
 



Null valued rows in Acronym: []
 



Null valued rows in Status: []
 



Null valued rows in Conditions: []
 



...

Null valued rows in Funder Type: [Row(Id='NCT00146315', Study Title='ESCAP: Supervised Exercise for Patients With Coronary Heart Disease in the Primary Care Setting', Acronym='', Status='COMPLETED', Conditions='Coronary Heart Disease', Interventions='BEHAVIORAL: Supervised exercise on a stationary bicycle, 3-5 days a week|BEHAVIORAL: Secondary prevention program for coronary heart disease', Sponsor='Basque Health Service', Collaborators='Preventive Services and Health Promotion Research Network|Castilla-La Mancha Health Service|Castilla-León Health Service|Dalt Sant Joan primary care center (Balears Islans Health Service)|Public Health Service of C', Enrollment=None, Funder Type=None, Type=None, Study Design=None, Start=None, Completion=None), Row(Id='taluña|Public Health Service of Madrid|Public Health Service of Galicia|Cantabria Health Service', Study Title='97.0', Acronym='OTHER_GOV', Status='INTERVENTIONAL', Conditions='Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: 

Null valued rows in Type: [Row(Id='NCT00146315', Study Title='ESCAP: Supervised Exercise for Patients With Coronary Heart Disease in the Primary Care Setting', Acronym='', Status='COMPLETED', Conditions='Coronary Heart Disease', Interventions='BEHAVIORAL: Supervised exercise on a stationary bicycle, 3-5 days a week|BEHAVIORAL: Secondary prevention program for coronary heart disease', Sponsor='Basque Health Service', Collaborators='Preventive Services and Health Promotion Research Network|Castilla-La Mancha Health Service|Castilla-León Health Service|Dalt Sant Joan primary care center (Balears Islans Health Service)|Public Health Service of C', Enrollment=None, Funder Type=None, Type=None, Study Design=None, Start=None, Completion=None), Row(Id='taluña|Public Health Service of Madrid|Public Health Service of Galicia|Cantabria Health Service', Study Title='97.0', Acronym='OTHER_GOV', Status='INTERVENTIONAL', Conditions='Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: SINGLE 

Null valued rows in Study Design: [Row(Id='NCT00146315', Study Title='ESCAP: Supervised Exercise for Patients With Coronary Heart Disease in the Primary Care Setting', Acronym='', Status='COMPLETED', Conditions='Coronary Heart Disease', Interventions='BEHAVIORAL: Supervised exercise on a stationary bicycle, 3-5 days a week|BEHAVIORAL: Secondary prevention program for coronary heart disease', Sponsor='Basque Health Service', Collaborators='Preventive Services and Health Promotion Research Network|Castilla-La Mancha Health Service|Castilla-León Health Service|Dalt Sant Joan primary care center (Balears Islans Health Service)|Public Health Service of C', Enrollment=None, Funder Type=None, Type=None, Study Design=None, Start=None, Completion=None), Row(Id='taluña|Public Health Service of Madrid|Public Health Service of Galicia|Cantabria Health Service', Study Title='97.0', Acronym='OTHER_GOV', Status='INTERVENTIONAL', Conditions='Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking:

Null valued rows in Start: [Row(Id='NCT00146315', Study Title='ESCAP: Supervised Exercise for Patients With Coronary Heart Disease in the Primary Care Setting', Acronym='', Status='COMPLETED', Conditions='Coronary Heart Disease', Interventions='BEHAVIORAL: Supervised exercise on a stationary bicycle, 3-5 days a week|BEHAVIORAL: Secondary prevention program for coronary heart disease', Sponsor='Basque Health Service', Collaborators='Preventive Services and Health Promotion Research Network|Castilla-La Mancha Health Service|Castilla-León Health Service|Dalt Sant Joan primary care center (Balears Islans Health Service)|Public Health Service of C', Enrollment=None, Funder Type=None, Type=None, Study Design=None, Start=None, Completion=None), Row(Id='taluña|Public Health Service of Madrid|Public Health Service of Galicia|Cantabria Health Service', Study Title='97.0', Acronym='OTHER_GOV', Status='INTERVENTIONAL', Conditions='Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: SINGLE

Null valued rows in Completion: [Row(Id='NCT00146315', Study Title='ESCAP: Supervised Exercise for Patients With Coronary Heart Disease in the Primary Care Setting', Acronym='', Status='COMPLETED', Conditions='Coronary Heart Disease', Interventions='BEHAVIORAL: Supervised exercise on a stationary bicycle, 3-5 days a week|BEHAVIORAL: Secondary prevention program for coronary heart disease', Sponsor='Basque Health Service', Collaborators='Preventive Services and Health Promotion Research Network|Castilla-La Mancha Health Service|Castilla-León Health Service|Dalt Sant Joan primary care center (Balears Islans Health Service)|Public Health Service of C', Enrollment=None, Funder Type=None, Type=None, Study Design=None, Start=None, Completion=None), Row(Id='taluña|Public Health Service of Madrid|Public Health Service of Galicia|Cantabria Health Service', Study Title='97.0', Acronym='OTHER_GOV', Status='INTERVENTIONAL', Conditions='Allocation: RANDOMIZED|Intervention Model: PARALLEL|Masking: S

In [20]:
df.write.format('csv').option('header',True).option('delimiter', ',').save('clinicaltrial_2023_cleaned_data.csv')

In [21]:
def check_null_values(df):
  """
  This function iterates through each column and prints the number of null values.
  """
  # Get all column names
  column_names = df.columns

  # Iterate through columns and count null values
  for col_name in column_names:
    null_count = df.filter(F.col(col_name).isNull()).count()
    print(f"Column: {col_name}, Null Values: {null_count}")

# Example usage
check_null_values(df)

Column: Id, Null Values: 0


Column: Study Title, Null Values: 0


Column: Acronym, Null Values: 0


Column: Status, Null Values: 0


Column: Conditions, Null Values: 0


...

Column: Funder Type, Null Values: 2


Column: Type, Null Values: 2


Column: Study Design, Null Values: 2


Column: Start, Null Values: 2


Column: Completion, Null Values: 2


In [22]:
display(df)

DataFrame[Id: string, Study Title: string, Acronym: string, Status: string, Conditions: string, Interventions: string, Sponsor: string, Collaborators: string, Enrollment: string, Funder Type: string, Type: string, Study Design: string, Start: string, Completion: string]

In [23]:
df.write.format("com.databricks.spark.csv").save("file:///home//download//datacsv")

24/03/07 17:34:33 ERROR FileOutputCommitter: Mkdirs failed to create file:/home/download/datacsv/_temporary/0
24/03/07 17:34:34 ERROR Executor: Exception in task 14.0 in stage 150.0 (TID 1052)
java.io.IOException: Mkdirs failed to create file:/home/download/datacsv/_temporary/0/_temporary/attempt_20240307173433920426583035832744_0150_m_000014_1052 (exists=false, cwd=file:/run/media/i666sapple/VOLUME2/VS CODE/Project_personal)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1064)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStream(CodecStreams.scala:81)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStreamWriter(CodecStreams.scala:92)
	at

24/03/07 17:34:34 ERROR FileFormatWriter: Aborting job 336f664c-bcd1-4bc6-b585-df69b72b36a4.
org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 150.0 failed 1 times, most recent failure: Lost task 7.0 in stage 150.0 (TID 1045) (1666sapple executor driver): java.io.IOException: Mkdirs failed to create file:/home/download/datacsv/_temporary/0/_temporary/attempt_20240307173433920426583035832744_0150_m_000007_1045 (exists=false, cwd=file:/run/media/i666sapple/VOLUME2/VS CODE/Project_personal)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1064)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStream(CodecStreams.scala:81)
	at org.apache.s

Py4JJavaError: An error occurred while calling o421.save.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 7 in stage 150.0 failed 1 times, most recent failure: Lost task 7.0 in stage 150.0 (TID 1045) (1666sapple executor driver): java.io.IOException: Mkdirs failed to create file:/home/download/datacsv/_temporary/0/_temporary/attempt_20240307173433920426583035832744_0150_m_000007_1045 (exists=false, cwd=file:/run/media/i666sapple/VOLUME2/VS CODE/Project_personal)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:515)
	at org.apache.hadoop.fs.ChecksumFileSystem.create(ChecksumFileSystem.java:500)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1195)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1175)
	at org.apache.hadoop.fs.FileSystem.create(FileSystem.java:1064)
	at org.apache.spark.sql.execution.datasources.CodecStreams$.createOutputStream(CodecStreams.scala:81)
	at org.apache.spark.sql.execution.datasources.Co

In [26]:
data =df.toPandas()

In [28]:
data.to_csv('clinicaltrial_2023_cleaned_data.csv', index=False)

: 

# PySpark Dataframe analysis


Q1: Number of Studies in the dataset?

In [ ]:
num_unique_items = df.select("Study Title").distinct().count()

print(f"Number of unique Studies in the dataset are: {num_unique_items}")


Number of unique Studies in the dataset are: 481102


In [ ]:
num_items = df.select("Study Title").count()

print(f"Number of  Studies in the dataset are: {num_items}")


Number of  Studies in the dataset are: 483422


In [ ]:
row = df.select("Study Title")
distict_data = row.distinct()

duplicate_data = row.exceptAll(distict_data)
display(duplicate_data)


Study Title
Computerized Assessment for Patients With Cancer
Study to Evaluate the Safety and Efficacy of CJ-12420 in Patients With Erosive Esophagitis
Endometrial Preparation in Frozen Embryo Transfer Cycles
Topical Antimicrobial Effectiveness Testing
Medication Maintenance Therapy in Community Pharmacy Settings
Study to Evaluate the Safety and Efficacy of VIRAZOLE® in Hospitalized Adult Participants With Respiratory Distress Due to COVID-19
A Study to Evaluate the Safety and Effectiveness of Asoprisnil in the Treatment of Uterine Fibroids
Efficacy of Oropharyngeal Exercises for Patients With Obstructive Sleep Apnea Using Mandibular Advancement Device
Efficacy and Safety of Diclofenac Sodium Gel in Hand Osteoarthritis
Tryptophan Metabolism in Human Brain Tumors


In [ ]:
display(duplicate_data)

Study Title


In [ ]:
df.select("Id").distinct().count()

Out[95]: 483422

In [ ]:
clinicaltrial_df = spark.read.csv("/FileStore/tables/clinicaltrial_2023_cleaned_data.csv", header=True, inferSchema=True)

clinicaltrial_df.createOrReplaceTempView("clinicaltrial_temp_view")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1118779165298587>:1
----> 1 clinicaltrial_df = spark.read.csv("/FileStore/tables/clinicaltrial_2023_cleaned_data.csv", header=True, inferSchema=True)
      3 clinicaltrial_df.createOrReplaceTempView("clinicaltrial_temp_view")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:729, in DataFrameReader.csv(self, path, schema, sep, encoding, quote, escape, comment, header, inferSchema, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, nullValue, nanValue

In [ ]:
sql_df = spark.sql("SELECT COUNT(DISTINCT Study_ID) AS Study_Title FROM clinicaltrial_2023_all_cleaned_data.csv")
distinct_studies_count = sql_df.collect()[0]["Study_Title"]
print(f"Number of distinct studies: {distinct_studies_count}")

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-1118779165298588>:1
----> 1 df = spark.sql("SELECT COUNT(DISTINCT Study_ID) AS Study_Title FROM clinicaltrial_2023_all_cleaned_data.csv")
      2 distinct_studies_count = df.collect()[0]["Study_Title"]
      3 print(f"Number of distinct studies: {distinct_studies_count}")

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/session.py:1387, in SparkSession.sql(self, sqlQuery, args, **kwargs)
   1385 try:
   1386     litArgs = {k: _to_java_column(lit(v)) for k, v in (